### Setup

In [1]:
from src.idspy.core.pipeline import Pipeline, PipelineEvent, hook, FitAwarePipeline
from src.idspy.core.state import State
from src.idspy.core.step import Step, FitAwareStep, Repeat

### Example `Steps`

In [2]:
class Load(Step):
    def __init__(self): super().__init__(provides=["data"])

    def run(self, state: State) -> None:
        state["data"] = [1, 2, 3]


class Sum(Step):
    def __init__(self): super().__init__(requires=["data"], provides=["sum"])

    def run(self, state: State) -> None:
        state["sum"] = sum(state["data"])


class MeanCenter(FitAwareStep):
    def __init__(self): super().__init__(requires=["data"], provides=["data"])

    def fit_impl(self, state: State) -> None:
        xs = state["data"];
        state["preproc.mean"] = sum(xs) / len(xs)

    def run(self, state: State) -> None:
        m = state["preproc.mean"]
        state["data"] = [x - m for x in state["data"]]


class Accumulate(Step):
    def __init__(self): super().__init__(requires=["sum"], provides=["tot"])

    def run(self, state: State) -> None:
        state["tot"] = state.get("tot", 0) + state["sum"]



### Build a `Pipeline` with Custom `@decorator` Hooks

In [3]:
class MyPipeline(Pipeline):
    @hook(PipelineEvent.PIPELINE_START)
    def _start(self, state: State) -> None:
        print("[pipeline] start")

    @hook(PipelineEvent.BEFORE_STEP, priority=-1)
    def _before(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline] before {index}: {step.name}")

    @hook(PipelineEvent.AFTER_STEP)
    def _after(self, step: Step, state: State, *, index: int) -> None:
        print(f"[pipeline] after {index}:  {step.name}")

    @hook(PipelineEvent.PIPELINE_END)
    def _end(self, state: State) -> None:
        print("[pipeline] end")


s = State()
p = MyPipeline([Load(), Sum()], name="Plain")
p(s)
print(s.to_dict())
# [pipeline] start
# [pipeline] before 0: Load
# [pipeline] after 0:  Load
# [pipeline] before 1: Sum
# [pipeline] after 1:  Sum
# [pipeline] end
# {'data': [1, 2, 3], 'sum': 6}

[pipeline] start
[pipeline] before 0: Load
[pipeline] after 0:  Load
[pipeline] before 1: Sum
[pipeline] after 1:  Sum
[pipeline] end
{'data': [1, 2, 3], 'sum': 6}


### Build a `FitAwarePipeline` that fits all `FitAwareSteps` prior to execution

In [4]:
s = State({"data": [1.0, 2.0, 3.0]})
fp = FitAwarePipeline([MeanCenter(), Sum()], name="FitPipe", refit=False)
fp(s)
print(s.to_dict())
# {'data': [-1.0, 0.0, 1.0], 'preproc.mean': 2.0, 'sum': 0.0}

# secondo run senza refit (usa ancora mean=2.0)
s["data"] = [2.0, 4.0, 6.0]
fp(s)
print(s.to_dict())
# {'data': [0.0, 2.0, 4.0], 'preproc.mean': 2.0, 'sum': 6.0}

# pipeline con refit=True
s2 = State({"data": [2.0, 4.0, 6.0]})
fp_refit = FitAwarePipeline([MeanCenter(), Sum()], name="FitPipeRefit", refit=True)
fp_refit(s2)
print(s2.to_dict())
# {'data': [-2.0, 0.0, 2.0], 'preproc.mean': 4.0, 'sum': 0.0}

{'data': [-1.0, 0.0, 1.0], 'preproc.mean': 2.0, 'sum': 0.0}
{'data': [0.0, 2.0, 4.0], 'preproc.mean': 2.0, 'sum': 6.0}
{'data': [-2.0, 0.0, 2.0], 'preproc.mean': 4.0, 'sum': 0.0}


### Repeat the Pipeline `count` times

In [5]:
p = Pipeline([Load(), Sum(), Accumulate()])
rp = Repeat(p, count=3)
s = State()
rp(s)
print(s.to_dict())
# 1° iter: data=[1,2,3], sum=6, tot=6
# 2° iter: data=[1,2,3], sum=6, tot=12
# 3° iter: data=[1,2,3], sum=6, tot=18
# {'data': [1, 2, 3], 'sum': 6, 'tot': 18}

{'data': [1, 2, 3], 'sum': 6, 'tot': 18}
